In [ ]:
!pip install "transformers==4.34.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.23.0" "bitsandbytes==0.41.1" "trl==0.4.7" "safetensors>=0.3.1" ipywidgets wandb --upgrade
!python -c "import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'"
!pip install ninja packaging
!MAX_JOBS=1 pip install flash-attn --no-build-isolation

In [5]:
# add parent directory to path
import sys
import os

project_dir = os.getcwd()
parent_dir = os.path.dirname(project_dir)
sys.path.insert(0, parent_dir)

In [6]:
# configs
from huggingface_hub import notebook_login
import wandb

model_id = "meta-llama/Llama-2-7b-chat-hf"
output_dir = "llama-7-chat-instruction-int4-fc-op_glaive-sft"
hub_model_id = f"SebastianS/{output_dir}"
wandb.init(project="function_calling", id=output_dir, entity="sebastiansosa", notes="Trained for 3 epochs on shuffled data from openplugin and glaive datasets with a train test split of 99.7% train.")
wandb.define_metric("fc_combine", goal="maximize")
wandb.define_metric("exact_match", goal="maximize")
wandb.define_metric("perplexity", goal="minimize")
wandb.define_metric("epoch")
notebook_login()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [7]:
# get data and convert to Datasets format
import pandas as pd
from datasets import Dataset, DatasetDict
from gaitor_function_calling.data.data_utils import DataAbstractor
from gaitor_function_calling.data.prompting_utils import INSTRUCTION

data_abstractor = DataAbstractor("op_glaive_10k.json", "full_sft")
# train, test = data_abstractor.build_data(INSTRUCTION, True, .997)
# data_abstractor.save(data_abstractor.raw_data, train, test)

train_df = pd.DataFrame(data_abstractor.train_data)
test_df = pd.DataFrame(data_abstractor.test_data)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9820
    })
    test: Dataset({
        features: ['text'],
        num_rows: 30
    })
})

In [8]:
# setup metrics and evaluation
import json
from transformers import EvalPrediction
from gaitor_function_calling.evaluation.evaluation_utils import FunctionCallingMetric, compute_perplexity, get_logits_and_labels
from gaitor_function_calling.data.prompting_utils import INSTRUCTION, json_arguments_from_prompt, generate_prediction
import numpy as np

fc_metric = FunctionCallingMetric()

def custom_evaluation(eval_dataset, model, tokenizer, epoch, save_prompts_path=False):
    print("Starting custom evaluation.")
    model.eval()  # Set the model to evaluation mode
    results = {}

    fc_results = []
    exact_match_results = []
    perplexity_results = []


    if save_prompts_path:
        try:
            with open(save_prompts_path, "r") as f:
                dpo_data = json.load(f)
        except:
            dpo_data = []


    for idx, example in enumerate(eval_dataset):
        print(f"Example {idx}: ", end="")
        post_message = ""
        
        # Custom Function Calling metric
        prompts = None
        try:
            generated_str = generate_prediction(example["text"], model, tokenizer, INSTRUCTION)
            generated_arguments, expected_arguments, prompts = json_arguments_from_prompt(
                example["text"],
                generated_str,
                INSTRUCTION,
                {"idx": idx, "epoch": epoch}
            )
            fc_result = fc_metric.run(generated_arguments, expected_arguments)

            fc_results.append(fc_result)

            if save_prompts_path:
                # add prompts to dpo_data.json
                dpo_data.append({
                    "fc_result": fc_result,
                    **prompts
                })
                with open(save_prompts_path, "w") as f:
                    json.dump(dpo_data, f)
        except Exception as e:
            post_message += f"Error function calling: {e}\n"
            fc_results.append(0)

        # exact match metric
        if prompts:
            exact_match_res = fc_metric._sentence_similarity(prompts["expected_str"].split("[/INST]")[1], prompts["generated_str"].split("[/INST]")[1])
            exact_match_results.append(exact_match_res)
        else:
            exact_match_results.append(0)

        # perplexity metric
        try:
            logits, labels = get_logits_and_labels(example["text"], model, tokenizer)
            perplexity = compute_perplexity(logits[..., :-1, :], labels).item()
            perplexity_results.append(perplexity)
        except Exception as e:
            post_message += f"Error perplexity: {e}\n"
            # perplexity_results.append(float('inf'))

        example_metric = {
            "fc_combine": fc_results[-1],
            "exact_match": exact_match_results[-1],
            "perplexity": perplexity_results[-1] if len(perplexity_results) else None
        }
        print(example_metric)
        if post_message:
            print(post_message)
        
    results["fc_combine"] =  sum(fc_results) / len(fc_results)
    results["exact_match"] =  sum(exact_match_results) / len(exact_match_results)
    results["perplexity"] =  sum(perplexity_results) / len(perplexity_results) if len(perplexity_results) else 1

    return results

In [9]:
# base LLM model and tokenizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=False,
    device_map="auto",
)
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# peft config on the model
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)


In [11]:
# training config
from transformers import TrainingArguments
from trl import SFTTrainer


args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=3,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    report_to="wandb",
    push_to_hub=True,  # Enable pushing to Hub
    hub_model_id=hub_model_id,  # Hugging Face Hub model ID
    hub_strategy="every_save",  # Push to Hub every epoch
    disable_tqdm=True # disable tqdm since with packing values are in correct
)


max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_dict["train"],
    # eval_dataset=dataset_dict["test"],
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    dataset_text_field="text",
    args=args,
)

Map:   0%|          | 0/9820 [00:00<?, ? examples/s]

In [13]:
others = {
            "fc_combine": ,
            "exact_match": exact_match_results[-1],
            "perplexity": perplexity_results[-1] if len(perplexity_results) else None
        }
wandb.log({"epoch": epoch,

In [12]:
# Training and Evaluation Loop
save_eval_path = "./data/prompts/"+os.path.basename(data_abstractor.paths["train_dir"]+".json")
eval_result = custom_evaluation(dataset_dict["test"], model, tokenizer, -1, save_eval_path)
print(f"Evaluation results for epoch {-1}: {eval_result}")
wandb.log({"epoch": -1, **eval_result})

num_epochs = 3
for epoch in range(num_epochs):
    print(f"Begin epoch {epoch}")
    # Train for one epoch
    trainer.train()

    # Evaluate the model with the custom evaluation logic
    eval_result = custom_evaluation(dataset_dict["test"], model, tokenizer, epoch, save_eval_path)

    wandb.log({"epoch": epoch, **eval_result})

    # Save the model after each epoch (optional)
    trainer.save_model()

    

    # Log or print evaluation results
    print(f"Evaluation results for epoch {epoch}: {eval_result}\n\n\n")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Starting custom evaluation.
Example 0: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 7.66923189163208}
Error function calling: No function call found in generated data

Example 1: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 6.339871883392334}
Error function calling: No function call found in generated data

Example 2: {'fc_combine': 1.0, 'exact_match': 0.7547522187232971, 'perplexity': 9.583250999450684}
Example 3: {'fc_combine': 1.0, 'exact_match': 0.8241101503372192, 'perplexity': 7.283013820648193}
Example 4: {'fc_combine': 1.0, 'exact_match': 0.8586133122444153, 'perplexity': 8.35544490814209}
Example 5: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 8.844438552856445}
Error function calling: No function call found in generated data

Example 6: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 6.977672100067139}
Error function calling: No function call found in generated data

Example 7: {'fc_combine': 0, 'exact_match': 0, 'perplexity': 5.191576957702637}
Error functio

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.5622, 'learning_rate': 0.0002, 'epoch': 0.01}
{'loss': 0.8084, 'learning_rate': 0.0002, 'epoch': 0.01}
{'loss': 0.4028, 'learning_rate': 0.0002, 'epoch': 0.02}
{'loss': 0.2104, 'learning_rate': 0.0002, 'epoch': 0.02}
{'loss': 0.2042, 'learning_rate': 0.0002, 'epoch': 0.03}
{'loss': 0.1583, 'learning_rate': 0.0002, 'epoch': 0.04}
{'loss': 0.1856, 'learning_rate': 0.0002, 'epoch': 0.04}
{'loss': 0.1575, 'learning_rate': 0.0002, 'epoch': 0.05}
{'loss': 0.1854, 'learning_rate': 0.0002, 'epoch': 0.05}
{'loss': 0.1256, 'learning_rate': 0.0002, 'epoch': 0.06}
{'loss': 0.1418, 'learning_rate': 0.0002, 'epoch': 0.07}
{'loss': 0.1351, 'learning_rate': 0.0002, 'epoch': 0.07}
{'loss': 0.1156, 'learning_rate': 0.0002, 'epoch': 0.08}
{'loss': 0.1169, 'learning_rate': 0.0002, 'epoch': 0.09}
{'loss': 0.1811, 'learning_rate': 0.0002, 'epoch': 0.09}
{'loss': 0.156, 'learning_rate': 0.0002, 'epoch': 0.1}
{'loss': 0.1086, 'learning_rate': 0.0002, 'epoch': 0.1}
{'loss': 0.0897, 'learning_rate': 

KeyboardInterrupt: 

In [ ]:
wandb.finish()